## src/data
- data_loader.py
- preprocessor.py
- tokenizer_wrapper.py


#### preprocessor.py
- parse_problems_column
    - problem 열을 flatten
- add_choices_len
    - choices_len 추가


In [1]:
import pandas as pd
import ast

def parse_problems_column(df: pd.DataFrame) -> pd.DataFrame:
    if 'problems' not in df.columns:
        return df

    df['problems'] = df['problems'].apply(ast.literal_eval)
    problems_df = df['problems'].apply(pd.Series)

    df = pd.concat([df.drop(columns=['problems']), problems_df], axis=1)

    return df


def add_choices_len(df: pd.DataFrame) -> pd.DataFrame:
    if 'choices' in df.columns:
        df['choices_len'] = df['choices'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    return df

In [15]:
### TEST
import os

ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
df = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
df.info()


df_parse = parse_problems_column(df)

df_add_choices = add_choices_len(df_parse)

df_add_choices.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


,id,paragraph,question_plus,question,choices,answer,choices_len
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",NaN,상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,4
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",NaN,(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,4
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,NaN,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,4
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,NaN,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,4
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",NaN,(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,4


#### data_loader.py
- load_data()
    - csv 로드, Flatten, choices_len 추가
- 전처리 함수
- hf dataset 만들기.

In [ ]:
from 


def load_csv(data_path: str) -> pd.DataFrame:
    return pd.read_csv(data_path)


def preprocess_dataframe(df:pd.DataFrame) -> pd.DataFrame:
    df = parse_problems_column(df)
    df = add_choices_len(df)
    return df




## src/prompt
- prompt_builder.py
- prompt_registry.py
- templates/
    system/
    user/

#### prompt_registry.py
- template(.txt)를 여러 버전으로 조회해서 문자열을 반환

In [8]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Optional, Tuple
from collections import defaultdict


class PromptRegistry:
    def __init__(
        self,
        templates_dir: Optional[str | Path] = None,
        verbose: bool = False
    ):
        if templates_dir is None:
            self.templates_dir = Path(__file__).parent / "templates"
        else:
            self.templates_dir = Path(templates_dir)
        
        self.templates: Dict[str, str] = {}
        self.verbose = verbose

        self._load_templates()

    def _load_templates(self) -> None:
        if not self.templates_dir.exists():
            raise FileNotFoundError(f"templates_dir 경로가 존재하지 않습니다: {self.templates_dir}")

        candidates = defaultdict(dict)

        for file_path in self.templates_dir.rglob("*.txt"):
            parsed = self._parse_filename(file_path)
            if parsed is None:
                continue
            
            role, choices_len, p_type, name = parsed

            try:
                content = file_path.read_text(encoding="utf-8")
            except Exception:
                if self.verbose:
                    print(f"파일 읽기 실패: {file_path}")
                continue

            key = f"{role}/{file_path.name}"
            self.templates[key] = content

            if role == "user":
                base_key = (role, choices_len, name)
                candidates[base_key][p_type] = key
        
        self._validate_user_pairs(candidates)

        if self.verbose:
            system_cnt = sum(k.startswith("system/") for k in self.templates)
            user4_cnt = sum(k.startswith("user/4_") for k in self.templates)
            user5_cnt = sum(k.startswith("user/5_") for k in self.templates)
            print(f"template loading 완료: system={system_cnt}, user_4={user4_cnt}, user_5={user5_cnt}")

    def _parse_filename(self, file_path: Path) -> Optional[Tuple[str, str, str, str]]:
        """
        파일명 규칙: {role}/{choices_len}_{type}_{name}.txt
        - role: system/user
        - choices_len: 4/5
        - type: user의 경우 question/ no_question
        - name: 이름
        """
        
        # 규칙에 맞는것만 가져오기. -> 맞지 않는경우에는? -> if verbose: 라면 print로 {file_name}은 안맞다고 하고 다음으로 넘어가는걸로.
        role = file_path.parent.name
        stem = file_path.stem
        parts = stem.split('_')

        if role == "system":
            if len(parts) == 2:
                choices_len, name = parts
                return role, choices_len, "default", name
        
        elif role == "user":
            if len(parts) == 3:
                choices_len, p_type, name = parts
                if p_type in ("question", "no_question"):
                    return role, choices_len, p_type, name
        
        if self.verbose:
            print(f"규칙에 맞지 않는 파일명입니다. {file_path.name}")
        return None
    
    def _validate_user_pairs(self, candidates: dict) -> None:
        """
        user의 경우 'question', 'no_question' 쌍이 모두 존재하는지 검증
        """

        for (role, choices_len, name), type_map in candidates.items():
            if role != "user":
                continue

            has_q = "question" in type_map
            has_not_q = "no_question" in type_map
        
            if has_q and has_not_q:
                continue
            
            if self.verbose:
                print(f"user pair 누락: choices_len={choices_len}, name={name}, have={list(type_map.keys())}")

            for key in type_map.values():
                self.templates.pop(key, None)

In [14]:
### TEST
import sys
from pathlib import Path

root = Path.cwd()
if not (root / "src").exists():
    root = root.parent.parent

sys.path.insert(0, str(root))
print("added to sys.path:", root)

from src.prompt.prompt_registry import PromptRegistry

registry = PromptRegistry(verbose=True)

added to sys.path: /data/ephemeral/pro-nlp-generationfornlp-nlp-13
template loading 완료: system=0, user_4=0, user_5=0


#### prompt_builder.py
- 데이터를 입력받아 필요한 prompt(message)를 생성
- policy에 따라 결정됨.

In [ ]:
policies = {
    "system": {4: "v1", 5: "v2"},
    "user":   {4: "v1", 5: "v1"},
}

In [18]:
from typing import Dict, Optional, Any
from pathlib import Path


class PromptBuilder:
    def __init__(
            self,
            policy: Dict[str, Dict[int, str]],
            templates_dir: Optional[str | Path] = None,
            mode: str = "train",
            verbose: bool = False,
    ):
        self.mode = mode
        self.policy = policy
        self.registry = PromptRegistry(templates_dir=templates_dir, verbose=verbose)

    def build_message(self, row: Dict) -> Dict[str, Any]:
        system_message = self._get_system_message(row)
        user_message = self._get_user_message(row)

        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]

        if self.mode == "train":
            assistant_message = self._get_assistant_message(row)
            messages.append({"role": "assistant", "content": assistant_message})

        return {
            "id": row.get("id"),
            "messages": messages,
            "label": int(row["answer"]),
        }

    def _get_system_message(self, row):
        choices_len = row["choices_len"]
        version = self.policy["system"][choices_len]
        key = f"system/{choices_len}_{version}.txt"

        template = self.registry.templates[key]
        return template

    def _get_user_message(self, row):
        choices_len = int(row["choices_len"])
        version = self.policy["user"][choices_len]

        paragraph = row["paragraph"]
        question = row["question"]
        choices = row["choices"]
        choices_str = self._format_choices(choices)

        q_plus = row.get("question_plus", None)
        has_plus = bool(q_plus) and str(q_plus) != "nan"
        p_type = "plus" if has_plus else "no_plus"
        key = f"user/{choices_len}_{p_type}_{version}.txt"

        template = self.registry.templates[key]

        if has_plus:
            return template.format(
                paragraph=paragraph,
                question_plus=q_plus,
                question=question,
                choices=choices_str,
            )
        return template.format(
            paragraph=paragraph,
            question=question,
            choices=choices_str,
        )

    def _get_assistant_message(row):
        return str(row['answer'])

    def _format_choices(self, choices: Any) -> str:
        if isinstance(choices, list):
            return "\n".join([f"{i+1}. {c}" for i, c in enumerate(choices)])
        return str(choices)

In [5]:
### TEST
import os
import sys
from pathlib import Path

nb_dir = Path(os.getcwd())

# 프로젝트 루트: notebooks/Jang -> notebooks -> project_root
project_root = nb_dir.parents[1]  # /data/ephemeral/pro-nlp-generationfornlp-nlp-13

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("project_root:", project_root)
print("sys.path[0]:", sys.path[0])

project_root: /data/ephemeral/pro-nlp-generationfornlp-nlp-13
sys.path[0]: /data/ephemeral/pro-nlp-generationfornlp-nlp-13


In [6]:
from src.prompt.prompt_registry import PromptRegistry
from src.prompt.prompt_builder import PromptBuilder
from src.data.preprocessor import parse_problems_column, add_choices_len


registry = PromptRegistry(verbose=True)
print("loaded templates:", len(registry.templates))

template loading 완료: system=2, user_4=2, user_5=2
loaded templates: 6


In [16]:
policy = {
    "system": {4: "v1", 5: "v1"},
    "user":   {4: "v1", 5: "v1"},
}

builder = PromptBuilder(
    mode="train",
    verbose=True,
    policy=policy
)

template loading 완료: system=2, user_4=2, user_5=2


In [17]:
builder.build_message(df_add_choices.iloc[3,:])

{'id': 'generation-for-nlp-428',
 'messages': [{'role': 'system',
   'content': "당신은 **지식 추론(Knowledge Inference) 전문가**입니다.\n이 유형은 정답이 지문에 그대로 쓰여 있지 않을 수 있으며, 지문은 '조건/단서'를 제공합니다.\n지문에서 주어진 조건을 정확히 반영하고, 그 조건과 모순되지 않는 범위에서 일반적으로 알려진 지식을 적용해 가장 타당한 선택지 하나를 고르십시오."},
  {'role': 'user',
   'content': '### 지문\n이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다. …(중략) …소정방이 신라군이 늦게 왔다는 이유로 군문에서 신라 독군 김문영의 목을 베고자 하니, 그가 군사들 앞에 나아가 “황산 전투를 보지도 않고 늦게 온 것을 이유로 우리를 죄 주려 하는구나. 죄도 없이 치욕을 당할 수는 없으니, 결단코 먼저 당나라 군사와 결전을 한 후에 백제를 쳐야겠다.”라고 말하였다.\n\n### 질문\n밑줄 친 ‘그’에 대한 설명으로 옳은 것은?\n\n### 선택지\n1. 살수에서 수의 군대를 물리쳤다 .\n2. 김춘추 의 신라 왕위 계승을 지원하였다 .\n3. 청해진을 설치하고 해상 무역을 전개하였다 .\n4. 대가야를 정벌하여 낙동강 유역을 확보하였다 .\n\n### 문제 해결 가이드라인\n1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)\n2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.\n3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.\n\n정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.\n정답:'},
  {'role': 'assistant', 'content': '2'}],
 'label': 2}

In [9]:
### TEST
import os
import pandas as pd

ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
df_test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))

df_pars_test = parse_problems_column(df_test)

df_add_choices_test = add_choices_len(df_pars_test)

df_add_choices_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     869 non-null    int64 
 1   id             869 non-null    object
 2   paragraph      869 non-null    object
 3   question       869 non-null    object
 4   choices        869 non-null    object
 5   answer         869 non-null    object
 6   question_plus  44 non-null     object
 7   choices_len    869 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 54.4+ KB


In [12]:
df_pars_test = parse_problems_column(df_test)

ValueError: malformed node or string: {'question': '윗글의 내용과 일치하지 않는  것은?', 'choices': ['같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다 .', '다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다 .', '독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를  매개로 접할 수 있다.', '독자의 배경지식,  관점 ,  읽기 환경 ,  과제는 독자의 의미 구성에   영향을 주는 독자 요인이다.', '독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을  받으며 필자와 간접적으로 대화한다.'], 'answer': ''}

In [13]:
df_test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
df_test.head()

,Unnamed: 0,id,paragraph,problems,question_plus
0,0,generation-for-nlp-0,사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다 . 독서의 즐거움에는 ...,"{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...",NaN
1,1,generation-for-nlp-1,사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다 . 독서의 즐거움에는 ...,{'question': '윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은...,NaN
2,2,generation-for-nlp-2,(가 ) 중국에서 비롯된 유서( 類書)는 고금의 서적에서 자료를 수집하고 항목별로...,"{'question': '(가 )와 (나 )에 대한 설명으로 가장 적절한 것은?',...",NaN
3,3,generation-for-nlp-3,(가 ) 중국에서 비롯된 유서( 類書)는 고금의 서적에서 자료를 수집하고 항목별로...,"{'question': '[A ]에 대한 이해로 적절하지 않은 것은?', 'cho...",NaN
4,4,generation-for-nlp-4,(가 ) 중국에서 비롯된 유서( 類書)는 고금의 서적에서 자료를 수집하고 항목별로...,"{'question': '㉮에 대한 이해를 바탕으로 ㉠ , ㉡에 대해 파악한 내용...",NaN


In [11]:
df_pars_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     869 non-null    int64 
 1   id             869 non-null    object
 2   paragraph      869 non-null    object
 3   question_plus  44 non-null     object
 4   question       869 non-null    object
 5   choices        869 non-null    object
 6   answer         869 non-null    object
 7   question_plus  44 non-null     object
 8   choices_len    869 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 61.2+ KB


In [10]:
policy = {
    "system": {4: "v1", 5: "v1"},
    "user":   {4: "v1", 5: "v1"},
}

builder = PromptBuilder(
    mode="test",
    verbose=True,
    policy=policy
)

template loading 완료: system=2, user_4=2, user_5=2


In [11]:
builder.build_message(df_add_choices_test.iloc[7,:])

{'id': 'generation-for-nlp-7',
 'messages': [{'role': 'system',
   'content': '당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.\n이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.\n당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.'},
  {'role': 'user',
   'content': '### 지문\n(가 ) 중국에서 비롯된 유서( 類書)는 고금의 서적에서 자료를  수집하고 항목별로 분류,  정리하여 이용에 편리하도록 편찬한   서적이다.  일반적으로 유서는 기존 서적에서 필요한 부분을   뽑아 배열할 뿐 상호 비교하거나 편찬자의 해석을 가하지  않았다.  유서는 모든 주제를 망라한 일반 유서와 특정 주제를   다룬  전문 유서로 나눌 수 있으며,  편찬 방식은 책에 따라   다른 경우가 많았다.  중국에서는 대체로 왕조 초기에 많은  학자를 동원하여 국가 주도로 대규모 유서를 편찬하여 간행 하였다.  이를 통해 이전까지의 지식을 집성하고 왕조의  위엄을 과시할 수 있었다. 고려 때 중국 유서를 수용한 이후,  조선에서는 중국 유서를   활용하는 한편,  중국 유서의 편찬 방식에 ⓐ따라  필요에  맞게 유서를 편찬하였다.  조선의 유서는 대체로 국가보다  개인이 소규모로 편찬하는 경우가 많았고 ,  목적에 따른 특정   주제의 전문 유서가 집중적으로 편찬되었다.  전문 유서  가운데 편찬자가 미상인 유서가 많은데,  대체로 간행을  염두에 두지 않고 기존 서적에서 필요한 부분을 발췌 ,  기록 하여 시문 창작 ,  과거 시험 등 개인적 목적으로 유서를 활용 하고자 하였기 때문이었다. 이 같은 유서 편찬 경향이 지속되는 가운데 17세기부터 실학의   학풍이 하나의 조류를 형성하면서 유서 편찬에 변화가 나타났다 .   ㉮실학자들의 유서 는 현실 개혁의 뜻을 담았고,  편찬 의도를  지식의 제공과 확산에 두었다.  또한 단순 정리

In [12]:
df_add_choices_test.iloc[7,:]

Unnamed: 0                                                       7
id                                            generation-for-nlp-7
paragraph        (가 ) 중국에서 비롯된 유서( 類書)는 고금의 서적에서 자료를  수집하고 항목별로...
question                            문맥상 ⓐ ～ⓔ와 바꾸어 쓰기에 적절하지 않은  것은?
choices          [ⓐ :의거(依據)하여, ⓑ :계몽(啓蒙)하는, ⓒ :용이(容易)하게, ⓓ :혼재(...
answer                                                            
question_plus                                                  NaN
choices_len                                                      5
Name: 7, dtype: object

In [13]:
df_add_choices_test.iloc[6,:]

Unnamed: 0                                                       6
id                                            generation-for-nlp-6
paragraph        (가 ) 중국에서 비롯된 유서( 類書)는 고금의 서적에서 자료를  수집하고 항목별로...
question         (가 ),  (나 )를 읽은 학생이 <보기>의 󰡔임원경제지 󰡕에 대해 보인  반응으...
choices          [현실 개혁의 뜻을 담았던 (가 )의 실학자들의 유서와 마찬가 지로 현실의 문제를 ...
answer                                                            
question_plus     서유구의 󰡔임원경제지 󰡕는 19세기까지의 조선과 중국 서적 들에서 향촌 관련 부분...
choices_len                                                      5
Name: 6, dtype: object

In [14]:
builder.build_message(df_add_choices_test.iloc[6,:])

{'id': 'generation-for-nlp-6',
 'messages': [{'role': 'system',
   'content': '당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.\n이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.\n당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.'},
  {'role': 'user',
   'content': '### 지문\n(가 ) 중국에서 비롯된 유서( 類書)는 고금의 서적에서 자료를  수집하고 항목별로 분류,  정리하여 이용에 편리하도록 편찬한   서적이다.  일반적으로 유서는 기존 서적에서 필요한 부분을   뽑아 배열할 뿐 상호 비교하거나 편찬자의 해석을 가하지  않았다.  유서는 모든 주제를 망라한 일반 유서와 특정 주제를   다룬  전문 유서로 나눌 수 있으며,  편찬 방식은 책에 따라   다른 경우가 많았다.  중국에서는 대체로 왕조 초기에 많은  학자를 동원하여 국가 주도로 대규모 유서를 편찬하여 간행 하였다.  이를 통해 이전까지의 지식을 집성하고 왕조의  위엄을 과시할 수 있었다. 고려 때 중국 유서를 수용한 이후,  조선에서는 중국 유서를   활용하는 한편,  중국 유서의 편찬 방식에 ⓐ따라  필요에  맞게 유서를 편찬하였다.  조선의 유서는 대체로 국가보다  개인이 소규모로 편찬하는 경우가 많았고 ,  목적에 따른 특정   주제의 전문 유서가 집중적으로 편찬되었다.  전문 유서  가운데 편찬자가 미상인 유서가 많은데,  대체로 간행을  염두에 두지 않고 기존 서적에서 필요한 부분을 발췌 ,  기록 하여 시문 창작 ,  과거 시험 등 개인적 목적으로 유서를 활용 하고자 하였기 때문이었다. 이 같은 유서 편찬 경향이 지속되는 가운데 17세기부터 실학의   학풍이 하나의 조류를 형성하면서 유서 편찬에 변화가 나타났다 .   ㉮실학자들의 유서 는 현실 개혁의 뜻을 담았고,  편찬 의도를  지식의 제공과 확산에 두었다.  또한 단순 정리

In [ ]:
# load_csv

# 

### 해야하는 동작
# .csv 불러오기.
# preprocessor.py 에 있는걸로 df 반환됨 (choices_len, parse_problems_column) 
# pd.DataFrame을 dataset으로 바꾸기.? 
# src/prompt 에 있는걸로 df 하나의 행을 프롬프트로 변경???
# 여기가 헷갈림. huggingface의 dataset으로 해야하는건지 아닌건지
# tokenize_wrapper 사용해서 -> -> -> 
# 최종 형태는 huggingface의 학습 가능한 형태의 dataset

# train_test_split도 여기서 해야하는건가...?
# 어디서 해야하는거지? 
# 아니면, 이건 따로 train.py에서 하는거고??
# train/test 를 따로 봐야하는건가



SyntaxError: invalid syntax (1201965725.py, line 20)

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union, Tuple

import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

from src.data.preprocessor import parse_problems_column, add_choices_len
from src.prompt.prompt_builder import PromptBuilder, PromptConfig
from src.data.tokenizer_wrapper import TokenizerWrapper, TokenizerConfig

@dataclass(frozen=True)
class DataConfig:
    train_path: Optional[Union[str, Path]] = None
    test_path: Optional[Union[str, Path]] = None
    valid_ratio: float = 0.1
    seed: int = 42

    do_split: bool = True


def load_csv(path: Union[str, Path]) -> pd.DataFrame:
    return pd.read_csv(path)


def build_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df = parse_problems_column(df)
    df = add_choices_len(df)

    return df


def df_to_dataset(df: pd.DataFrame) -> pd.DataFrame:
    return Dataset.from_pandas(df, preserve_index=False)


def make_train_valid_dataset(
    data_cfg: DataConfig,
    prompt_cfg: PromptConfig,
    tokenize_cfg_train: TokenizerConfig,
    tokenize_cfg_gen: TokenizerConfig,
    tokenizer,
) -> DatasetDict:
    
    df = load_csv(data_cfg.train_path)
    df = build_dataframe(df)

    if data_cfg.do_split:
        train_df, valid_df = train_test_split(
            df,
            test_size = data_cfg.valid_ratio,
            stratify=df["choices_len"],
            random_state = data_cfg.seed,
        )
        train_ds = Dataset.from_pandas(train_df, preserve_index=False)
        valid_ds = Dataset.from_pandas(valid_df, preserve_index=False)
    else:
        train_ds = Dataset.from_pandas(df, preserve_index=False)
        valid_ds = None
    
    prompt_cfg_train = PromptConfig(
        policy=prompt_cfg.policy,
        mode="train",
        templates_dir=prompt_cfg.templates_dir,
        verbose=prompt_cfg.verbose,
    )
    prompt_cfg_test = PromptConfig(
        policy=prompt_cfg.policy,
        mode="test",
        templates_dir=prompt_cfg.templates_dir,
        verbose=prompt_cfg.verbose,
    )

    builder_train = PromptBuilder(prompt_cfg_train)
    builder_test = PromptBuilder(prompt_cfg_train)

    tokenize_wrapper_train = TokenizerWrapper(tokenizer, tokenize_cfg_train)
    tokenize_wrapper_gen = TokenizerWrapper(tokenizer, tokenize_cfg_gen)

    train_msg = train_ds.map(
        builder_train.build_message,
        batched=False,
        remove_columns=train_ds.column_names,
        desc="Build train messages (teacher forcing)",
    )
    train_text = train_msg.map(
        tokenize_wrapper_train.to_text,
        batched=False,
        remove_columns=["messages"],
        desc="Serialize train to text",
    )
    train_tf = train_text.map(
        tokenize_wrapper_train.tokenize_fn,
        batched=True,
        remove_columns=["text"],
        desc="Tokenize train",
    )

    valid_msg = valid_ds.map(
        builder_test.build_message,
        batched=False,
        remove_columns=valid_ds.column_names,
        desc="Build valid messages (teacher forcing)",
    )
    valid_text = valid_msg.map(
        tokenize_wrapper_train.to_text,
        batched=False,
        remove_columns=["messages"],
        desc="Serialize valid to text",
    )
    valid_tf = valid_text.map(
        tokenize_wrapper_train.tokenize_fn,
        batched=True,
        remove_columns=["text"],
        desc="Tokenize valid",
    )

    valid_gen_msg = valid_ds.map(
        builder_test.build_message,
        batched=False,
        desc="Build valid_gen messages (prompt only)",
    )

    # 이게 맞는건지 모르겠음. -> 일단 보류
    def _to_text_keep_meta(ex):
        out = tokenize_wrapper_gen.to_text(ex)  
        out["id"] = ex.get("id")
        out["answer"] = ex.get("answer")      
        out["choices_len"] = ex.get("choices_len")
        return out

    valid_gen_text = valid_gen_msg.map(
        _to_text_keep_meta,
        batched=False,
        # message 
        remove_columns=["messages"],
        desc="Serialize valid_gen to text (+meta)",
    )

    def _tokenize_keep_meta(batch):
        tok = tokenize_wrapper_gen.tokenize_fn({"text": batch["text"]})
        tok["id"] = batch["id"]
        tok["answer"] = batch["answer"]
        tok["choices_len"] = batch["choices_len"]
        return tok

    valid_gen = valid_gen_text.map(
        _tokenize_keep_meta,
        batched=True,
        remove_columns=valid_gen_text.column_names,
        desc="Tokenize valid_gen (+meta)",
    )

    return DatasetDict(
        {
            "train": train_tf,
            "validation": valid_tf,
            "validation_gen": valid_gen,
        }
    )

# 원래 하나의 행씩 했으니까 일단 이건 빼자. 
# def make_test_dataset(
#     data_cfg: DataConfig,
#     prompt_cfg: PromptConfig,
#     tokenize_cfg_gen: TokenizerConfig,
#     tokenizer,
# ) -> Dataset:
    
#     df = build_dataframe(load_csv(data_cfg.test_path))
#     ds = df_to_dataset(df)

#     test_prompt_cfg = PromptConfig(
#         policy=prompt_cfg.policy,
#         mode="test",
#         templates_dir=prompt_cfg.templates_dir,
#         verbose=prompt_cfg.verbose,
#     )
#     builder = PromptBuilder(test_prompt_cfg)

#     # test는 generation prompt True
#     tw = TokenizerWrapper(tokenizer, TokenizerConfig(
#         max_length=tokenize_cfg_gen.max_length,
#         padding=tokenize_cfg_gen.padding,
#         truncation=tokenize_cfg_gen.truncation,
#         add_generation_prompt=True,
#     ))

#     orig_cols = ds.column_names
#     ds_msg = ds.map(lambda ex: builder.build_message(ex), remove_columns=orig_cols)
#     ds_text = ds_msg.map(tw.to_text, remove_columns=["messages"])
#     ds_tok = ds_text.map(tw.tokenize_fn, batched=True, remove_columns=["text"])
#     return ds_tok

### 최종적으로 검토 필요

In [1]:
### TEST
import os
import sys
from pathlib import Path

nb_dir = Path(os.getcwd())

# 프로젝트 루트: notebooks/Jang -> notebooks -> project_root
project_root = nb_dir.parents[1]  # /data/ephemeral/pro-nlp-generationfornlp-nlp-13

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("project_root:", project_root)
print("sys.path[0]:", sys.path[0])

project_root: /data/ephemeral/pro-nlp-generationfornlp-nlp-13
sys.path[0]: /data/ephemeral/pro-nlp-generationfornlp-nlp-13


In [2]:
from src.data.data_loader import DataConfig, make_train_valid_dataset
from src.prompt.prompt_builder import PromptConfig
from src.data.tokenizer_wrapper import TokenizerConfig

In [3]:
from transformers import AutoTokenizer

MODEL_NAME = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

In [4]:
data_cfg = DataConfig(
    train_path=project_root / "data" / "train.csv",
    valid_ratio=0.1,
    seed=42,
    do_split=True,
)

policy = {
    "system": {4: "v1", 5: "v1"},
    "user":   {4: "v1", 5: "v1"},
}

prompt_cfg = PromptConfig(
    policy=policy,
    templates_dir=project_root / "src" / "prompt" / "templates",
    verbose=False,
    mode="train",  # 여기 값은 dataloader 내부에서 train/test로 다시 만들어 쓰게 되어있음
)

tokenize_cfg_train = TokenizerConfig(
    max_length=2048,
    truncation=True,
    padding=False,
    add_generation_prompt=False,
)

tokenize_cfg_gen = TokenizerConfig(
    max_length=2048,
    truncation=True,
    padding=False,
    add_generation_prompt=True,
)

In [5]:
ds = make_train_valid_dataset(
    data_cfg=data_cfg,
    prompt_cfg=prompt_cfg,
    tokenize_cfg_train=tokenize_cfg_train,
    tokenize_cfg_gen=tokenize_cfg_gen,
    tokenizer=tokenizer,
)

ds

Build train messages (teacher forcing):   0%|          | 0/1827 [00:00<?, ? examples/s]

Serialize train to text:   0%|          | 0/1827 [00:00<?, ? examples/s]

Tokenize train:   0%|          | 0/1827 [00:00<?, ? examples/s]

Build valid messages (teacher forcing):   0%|          | 0/204 [00:00<?, ? examples/s]

Serialize valid to text:   0%|          | 0/204 [00:00<?, ? examples/s]

Tokenize valid:   0%|          | 0/204 [00:00<?, ? examples/s]

Build valid_gen messages (prompt only):   0%|          | 0/204 [00:00<?, ? examples/s]

Serialize valid_gen to text (+meta):   0%|          | 0/204 [00:00<?, ? examples/s]

Tokenize valid_gen:   0%|          | 0/204 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1827
    })
    validation: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 204
    })
    validation_gen: Dataset({
        features: ['id', 'answer', 'choices_len', 'input_ids', 'attention_mask'],
        num_rows: 204
    })
})

In [13]:
### 확인용
from typing import Optional, List
import random

def _safe_decode(tokenizer, input_ids: List[int], skip_special_tokens: bool = False) -> str:
    # skip_special_tokens=False 추천: chat_template 특수토큰/role 토큰이 보이면 원인 파악이 쉬움
    return tokenizer.decode(input_ids, skip_special_tokens=skip_special_tokens)

def show_samples(
    ds,
    tokenizer,
    split_name: str,
    n: int = 3,
    seed: int = 42,
    skip_special_tokens: bool = False,
    head_chars: int = 800,
    tail_chars: int = 300,
):
    """
    ds: DatasetDict or Dataset
    split_name: "train" / "validation" / "validation_gen" (DatasetDict일 때)
    """
    if hasattr(ds, "keys"):  # DatasetDict
        split = ds[split_name]
    else:
        split = ds

    rng = random.Random(seed)
    idxs = rng.sample(range(len(split)), k=min(n, len(split)))

    print(f"\n=== [{split_name}] sample {len(idxs)} ===")
    for i, idx in enumerate(idxs, 1):
        ex = split[idx]
        input_ids = ex["input_ids"]
        attn = ex.get("attention_mask", None)

        text = _safe_decode(tokenizer, input_ids, skip_special_tokens=skip_special_tokens)

        print(f"\n--- #{i} idx={idx} ---")
        print("len(input_ids):", len(input_ids))

        # label/answer 확인
        if "label" in ex:
            print("label:", ex["label"])
        if "answer" in ex:
            print("answer:", ex["answer"])
        if "choices_len" in ex:
            print("choices_len:", ex["choices_len"])
        if "id" in ex:
            print("id:", ex["id"])

        # attention mask 간단 체크
        if attn is not None:
            print("len(attention_mask):", len(attn), "| attn sum:", sum(attn))

        # 텍스트 미리보기
        print("\n[decoded]")
        print(text)

# 사용 예시:
# ds = make_train_valid_dataset(...) 결과 DatasetDict
show_samples(ds, tokenizer, "train", n=3, skip_special_tokens=False)


=== [train] sample 3 ===

--- #1 idx=1309 ---
len(input_ids): 928
label: 1
id: generation-for-nlp-2150
len(attention_mask): 928 | attn sum: 928

[decoded]
<|im_start|>system
당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.
이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.
당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>
<|im_start|>user
### 지문
미국 제너럴모터스(GM)의 자동차도 해킹을 통해 원격조종이 가능하다는 주장이 제기됐다. 피아트 크라이슬러가 같은 우려로 140만대 자동차에 대해 리콜을 결정한 지 1주일 만이다.로이터통신은 30일(현지시간) “범죄 의도가 없어 ‘화이트 햇’으로 불리는 전문해커 새미 캄카가 해킹으로 GM자동차의 문을 강제로 열고 시동까지 거는 모습을 유튜브에 공개했다”며 “캄카는 운전자에게 GM의 원격조종용 모바일 애플리케이션(앱)을 사용하지 말라고 조언했다”고 보도했다.캄카가 시연한 해킹은 GM이 내놓은 온스타 시스템을 탑재한 차량을 대상으로 이뤄졌다. 온스타는 자동차와 스마트폰을 연결해 차량을 제어하는 시스템이다. GM의 온스타 모바일 앱을 스마트폰에 설치, 사용한 사람은 300만명이 넘는다. 로이터에 따르면 캄카는 “100달러(약 11만원)를 들여 제작한 별도의 단말기를 이용해 자동차와 스마트폰의 연결 과정에 침투했다”고 주장했다.이에 대해 GM은 성명을 통해 “캄카의 주장은 이미 회사 기술자들이 점검했던 것”이라며 “지금은 개선을 마친 상태”라고 해명했다. 하지만 캄카는 “GM 관계자들과 논의한 결과 회사 조치는 보안의 취약성을 해결할 수 없었다”고 반박하면서 “다음주 미국 라스베이거스에서 열리는 데프콘퍼런스에서 해킹 방법에 대한 원리를 설명하겠다”고 밝혔다.로이터는 “GM에 문제점을 해결했는

In [14]:
show_samples(ds, tokenizer, "validation", n=3, skip_special_tokens=False)


=== [validation] sample 3 ===

--- #1 idx=163 ---
len(input_ids): 1585
label: 1
id: generation-for-nlp-2474
len(attention_mask): 1585 | attn sum: 1585

[decoded]
<|im_start|>system
당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.
이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.
당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>
<|im_start|>user
### 지문
한동안 잠잠하던 유로존(유로화 사용 17개국) 경제 상황이 심상치 않다. 유럽중앙은행(ECB)은 7일(현지시간) 통화정책회의에서 올해 유로존 성장률을 -0.5%로 전망했다. 지난해 12월에 내놓은 전망치보다 0.1%포인트 낮췄다. 올해 1.8%로 예상한 물가상승률도 내년에 1.3% 수준으로 떨어질 것으로 전망했다. 유로존에 경제성장률과 물가가 장기간 동시에 침체되는 ‘일본식 디플레이션’이 올 것이라는 우려가 커지는 이유다.○유로존 ‘일본식 디플레이션’ 오나마리오 드라기 ECB 총재는 이날 통화정책회의에서 기준금리를 연 0.75%로 동결했다. “통화정책만으로 문제를 해결할 수 없다”는 이유에서다. 또 “하반기부터는 경제 상황이 나아질 것”이라며 낙관론을 폈다. 그러나 전문가들은 “ECB가 시장을 너무 낙관적으로 보고 있다”고 지적했다. 덴마크 단스크은행의 라스 크리스텐센 이코노미스트도 “어떤 방식으로든 통화량을 늘리지 않으면 유로존은 디플레이션에 빠질 것”이라고 지적했다. ECB는 내년 유로존 물가상승률을 0.6~2.0%로 전망했다. 평균 1.3%다. 일반적으로 물가상승률이 1% 선으로 떨어지면 심각한 디플레이션으로 여긴다. 유로존 실업률은 사상 최고인 11.9%다. 유로존 은행의 기업대출은 지난 6개월간 1000억유로 줄었다. 그만큼 돈이 돌지 않는다는 얘기다. 독일의 

In [15]:

show_samples(ds, tokenizer, "validation_gen", n=3, skip_special_tokens=False)


=== [validation_gen] sample 3 ===

--- #1 idx=163 ---
len(input_ids): 1578
answer: 1
choices_len: 5
id: generation-for-nlp-2474
len(attention_mask): 1578 | attn sum: 1578

[decoded]
<|im_start|>system
당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.
이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.
당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>
<|im_start|>user
### 지문
한동안 잠잠하던 유로존(유로화 사용 17개국) 경제 상황이 심상치 않다. 유럽중앙은행(ECB)은 7일(현지시간) 통화정책회의에서 올해 유로존 성장률을 -0.5%로 전망했다. 지난해 12월에 내놓은 전망치보다 0.1%포인트 낮췄다. 올해 1.8%로 예상한 물가상승률도 내년에 1.3% 수준으로 떨어질 것으로 전망했다. 유로존에 경제성장률과 물가가 장기간 동시에 침체되는 ‘일본식 디플레이션’이 올 것이라는 우려가 커지는 이유다.○유로존 ‘일본식 디플레이션’ 오나마리오 드라기 ECB 총재는 이날 통화정책회의에서 기준금리를 연 0.75%로 동결했다. “통화정책만으로 문제를 해결할 수 없다”는 이유에서다. 또 “하반기부터는 경제 상황이 나아질 것”이라며 낙관론을 폈다. 그러나 전문가들은 “ECB가 시장을 너무 낙관적으로 보고 있다”고 지적했다. 덴마크 단스크은행의 라스 크리스텐센 이코노미스트도 “어떤 방식으로든 통화량을 늘리지 않으면 유로존은 디플레이션에 빠질 것”이라고 지적했다. ECB는 내년 유로존 물가상승률을 0.6~2.0%로 전망했다. 평균 1.3%다. 일반적으로 물가상승률이 1% 선으로 떨어지면 심각한 디플레이션으로 여긴다. 유로존 실업률은 사상 최고인 11.9%다. 유로존 은행의 기업대출은 지난 6개월간 1000억유로 줄었다. 그만큼 